Dans la première partie de la pratique, nous avons vu comment scraper une page web,toutefois, dans le cadre de notre discipline, nous n'aurons pas les pages localement, elles seront sur des serveurs distants, il faudra donc faire des requêtes pour récupérer le contenu de ces pages.

Malgré ses nombreuses fonctionnalités ([voir la documentation de BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/)) BeautifulSoup ne permet pas de faire une requête vers une ressource distante. Pour ceci, nous n'aurons pas besoin d'installer un outil externe. Le langage Python possède un très grand nombre de bibliothèques par défaut. C'est "request" qui va nous servir à charger nos pages web, même si c'est natif à Python, il faut **impérativement** l'importer avant de l'utiliser avec la ligne suivante :
```import requests```

Utilisez donc la cellule suivante pour importer "requests" et "BeautifulSoup" (reprenez la ligne d'importation du fichier "main.ipynb" pour "BeautifulSoup")

In [1]:
# Importez "requests" et "BeautifulSoup" ici

Parfait "requests" et "BeautifulSoup" sont importés, nous pouvons commencer à scraper. Nous allons mettre tout ça en application sur des produits du site boulanger.fr, imaginons que nous souhations récupérer certains produits, disons les casques audio et récupérer les informations suivantes :
- Marque
- Nom du produit
- Prix
- Est-ce que le produit possède un rabais ?
- Le rabais du produit ?

 Voici l'url qui nous intéresse "https://www.boulanger.com/c/tous-les-casques-audio?numPage=1", si vous accédez à la page, vous verrez une mise en page semblable à ceci :
![](boulanger-audio.png)

Nous allons commencer avec une page pour nous assurer que notre code fonctionne correctement. Mais avant, il faut récupérer nos sélecteurs grâce à la console du navigateur (onglet "Elements/Inspecteur"). Complètez la liste suivante avec les sélecteurs associés :

- Marque :
- Nom du produit :
- Prix : 
- Est-ce que le produit possède un rabais ? : 
- Le rabais du produit ? :
- Le lien vers la page détail de l'article :
  - Concernant ce selecteur, on essayera d'aller plus loin dans notre scraping et récupérer uniquement l'id contenu dans l'url

Pensez à valider vos sélecteurs via la console du navigateur (onglet "console") grâce à la méthode "document.querySelector("votre-selecteur")"

Voilà, nous avons nos sélecteurs, il est désormais temps de charger notre page avec "requests" avec le code suivant. Nous allons donc faire la même chose que précédemment sauf que cette fois-ci nous allons stocker les résultats de la première page dans un tableau de dictionnaires et sauvegarder le tout dans un fichier (csv ou xls(x) au choix).

In [12]:
from bs4 import BeautifulSoup
import requests

# Vous pouvez choisir un autre URL
lien = "https://www.boulanger.com/c/tous-les-casques-audio?numPage=1"
response = requests.get(lien)

soup = BeautifulSoup(response.text, 'html.parser')

# Completez le code pour récupérer les données

Notre code est fonctionnel, maintenant nous pouvons scraper à plus haute échelle et donc récupérer toutes les données sur toutes les pages de notre catégorie. **Toutefois**, vu que nous sommes tous au même endroit, nous allons limiter notre scraping aux sept premières pages, car en scrapant tout tous en même temps sur la même adresse IP, on risque de votre notre adresse IP être bannie. 
Pour ce faire, il nous faudra une boucle pour changer de page à la fin de chaque scraping. Si on regarde l'URL attentivement, il y a le paramètre "numPage" qui semble indiquer le numéro de page, il faudra donc mettre à jour le paramètre entre chaque tour de boucle.

Cette action étant longue, mais effectuée rapidement par notre robot, il est fort probable que nos actions soient considérées comme étant suspectes par les serveurs du site et que notre adresse IP publique soit bannie au bout d'un moment.

Pour éviter ceci, nous allons mettre en place un délai entre chaque requête, nous allons utiliser les bibliotèques "time" et "random", ces bibliothèques sont également natives à Python, il n'y aura pas besoin d'utiliser conda en amont pour importer. Le code suivant permet d'instaurer une pause dont le temps est situé entre 30 et 55 secondes `time.sleep(random.randrange(30, 55))`, il faudra mettre ce code au début de sa première boucle. **Si votre code ne vous semble pas correct, demandez de l'aide avant de le lancer, nous ne sommes pas à l'abri d'une boucle infinie**.

Rappelons qu'il est possible de vérifier le délai entre chaque crawl grâce au robots.txt du site. Vous pouvez voir les données grâce au code suivant :
```python
from urllib.request import urlopen
with urlopen("site.tld/robots.txt") as stream:
    print(stream.read().decode("utf-8"))
```
Si vous ne trouvez pas de référence au délai entre chaque crawl, utilisez la valeur aléatoire utilisée plus haut avec les fonctions time et random.

Enfin pour limiter encore plus le risque de voir son IP être bannie, nous allons proxifier nos IP. Nous allons charger nos proxies contenus dans le fichier proxy_list.txt puis itérer entre chaque requête sur un proxy différent. Notez qu'avec l'utilisation des proxies on peut diminuer le délai entre chaque requête car chacunes de vos requêtes aura une adresse IP différente.

Pour réaliser le scraping complet, nous aurons besoin, au minimum, de deux boucles :
- Une boucle permettant d'itérer entre les pages (c'est là qu'on trouvera `time.sleep(random.randrange(30, 55))`)
- Une boucle itérant entre les produits de la page courante pour donc récupérer les données

- [Voir documentation des boucles en Python](https://www.w3schools.com/python/python_for_loops.asp)
- [Voir documentation sur les tableaux en Python](https://www.w3schools.com/python/python_arrays.asp)
- [Voir rappel sur les dictionnaires en Python](https://www.w3schools.com/python/python_dictionaries.asp)

Le but de notre scraping est de réaliser un tableau de dictionnaires contenant les clés suivantes :
- marque
- nomProduit
- prix
- aRabais
- pourcentageRabais
- id : l'id représente le nombre contenu dans l'url de l'article. Pour récupérer ce nombre, il vous faudra utiliser des regex (voir plus bas pour plus d'explications).


# Regex (Regular Expression)

Les Regexes ou "Expressions Régulières/Rationnelles" en français sont des outils très puissants permettant de récupérer certaines parties d'une chaîne de caractères à partir d'un motif défini. Par motif, on entend qu'on ne temps plus à chercher "toto" dans la phrase "toto va à plage" mais plutôt la double répétition de "to" dans la phrase suscitée.

Les regexes sont notamment utilisées pour s'assurer qu'une adresse e-mail est correcte et donc corresponde au pattern {nom}@{domaine}.{tld}.

Dans le cas de nos casques sur le site de boulanger, l'id est contenu dans la valeur de l'attribut "href" qui se présente sous la forme "/ref/XXXXXXX" où "XXXXXXX" est l'id qui nous intéresse.

Que remarquez-vous ?
- ?


Pour récupérer notre id, il faudra utiliser la regex suivante et importer "re" (déjà importé dans la cellule suivante). Le code pour récupérer l'id est le suivant :
```python
# Cherche-moi la répétition de sept chiffres
regex_id = re.compile(r'\d{7}') 
attr_href_val = "/ref/1445520"
# Prends-moi la première occurence trouvée
id_produit = regex_id.search(attr_href_val).group(0)
id_produit
```
Le code ci-dessus sera donc à réutiliser dans votre boucle.

In [7]:
import sys
# On n'oublie pas d'installer fake_useragent car 
# ce n'est pas un bibliothèque native à Python
# Il nous faudra également définir votre source pour nos packages grâce à la ligne 6 "!conda config..."

# !conda config --append channels conda-forge
# !conda install --yes --prefix {sys.prefix} fake-useragent

import random
import time
import re

from fake_useragent import UserAgent

import pandas as pd
import itertools as it

# Complétez le code ci-dessous de façon à récupérer le contenu des sept premières pages et 
# ensuite en enregistrez chaque entrée dans un tableau de dictionnaires (récupérez le code du notebook précédent) pour le scraping et 
# changez les paramètres avec les sélecteurs définis plus haut.
# N'oubliez pas de changer la valeur du paramètre "numPage" entre chaque tour de boucle

# Partie des proxies
list_proxies = pd.read_csv('proxies_list.txt', header=None)
list_proxies = list_proxies.values.tolist()
list_proxies = list(it.chain.from_iterable(list_proxies))
ua = UserAgent()

proxy_pool = it.cycle(list_proxies)

# A remplir avec la liste des URL
liste_pages = []

for page in liste_pages:
    proxy = next(proxy_pool)

    response = requests.get(
        page,
        proxies={"http": proxy, "https": proxy}, 
        headers={'User-Agent': ua.random}
    )
    # Vu que nous jouons sur les adresses IP,
    # nous nous permettons de diminuer le délai entre chaque crawl
    time.sleep(random.randrange(15, 25))

    # Continuez la logique ici

'1445520'

Nous avons maintenant un fichier de travail, vous pouvez maintenant d'exploiter ces données comme vu précédemment :
- Nettoyage
- Analyse
- Graphiques (+ questions)

# Pour aller plus loin
Vous pouvez faire le même exercice avec des données immobilières, donnée que vous aurez récupérez sur le site https://www.avendrealouer.fr/ ou un site de votre choix. A noter que si votre scraping ne retourne rien c'est certainement que vous essayez de charger des données asynchrones, il vous faudra utiliser en complément Selenium (voir 3-scraping-selenium.ipynb).